In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
URL = "https://www.voetbalstats.nl/listjaarere.php?seizoenid=02"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [3]:
td_elements = soup.find_all("td", class_="linkere")

In [4]:
wedstrijden = []
for td in td_elements:
    a_tag = td.find("a", href=True)
    if a_tag and a_tag["href"].startswith("opstelere"):
        wedstrijden.append(a_tag["href"])

wedstrijden = list(set(wedstrijden))
print(len(wedstrijden))

ids = [int(x.split("=", 1)[1]) for x in wedstrijden]

306


In [5]:
maanden = {"januari" : 1, "februari": 2, "maart": 3, "april": 4, "mei": 5, "juni": 6, "juli": 7, "augustus": 8, "september": 9, "oktober": 10, "november": 11, "december": 12}
weekdagen = {"maandag": 1, "dinsdag": 2, "woensdag": 3, "donderdag": 4, "vrijdag": 5, "zaterdag": 6, "zondag": 7}

def get_datum(datum):
    weekdag  = int(weekdagen[datum.split()[0]])
    dag = int(datum.split()[1])
    maand = int(maanden[datum.split()[2]])
    jaar = int(datum.split()[3])

    return date(jaar, maand, dag)

In [6]:
def get_minuut(doelpunt):
    minuut = doelpunt.strip().split(". ")[0]
    try:
        return int(minuut)
    except:
        return minuut


In [88]:
rows = []

for i in range(len(wedstrijden)):
    wedstrijd_URL = "https://www.voetbalstats.nl/" + wedstrijden[i]
    id = ids[i]

    page = requests.get(wedstrijd_URL)
    soup = BeautifulSoup(page.content, "html.parser")

    page_title = soup.title.string if soup.title else "No title found"
    thuis = page_title.split("-")[0].strip()
    uit = page_title.split("-")[1].split("(")[0].strip()
    print(i, page_title)

    td_elements2 = soup.find_all("td", class_="linkere")

    
    datum = ""
    for td in td_elements2:
        a_tag = td.find("a", href=True)
        if a_tag and a_tag["href"].startswith("listjaarere"):
            datum = get_datum(a_tag.contents[0])
    
    a_tags = soup.find_all("a")

    doelpunt_spelers = []
    penalty_spelers = []
    eigen_doelpunt_spelers = []

    for a_tag in a_tags:
        bal_tag = a_tag.find('img', src='./afbeeldingen/bal.jpg')
        balpen_tag = a_tag.find('img', src='./afbeeldingen/balpen.jpg')
        baled_tag = a_tag.find('img', src='./afbeeldingen/baled.jpg')
        if bal_tag:
            doelpunt_spelers.append(a_tag.contents[1])
        if balpen_tag:
            penalty_spelers.append(a_tag.contents[2])
        if baled_tag:
            eigen_doelpunt_spelers.append(a_tag.contents[2])
    
    for speler in doelpunt_spelers:
        minuut = get_minuut(speler.strip().split(". ")[0])
        naam = speler.strip().split(". ")[1]
        rows.append([id, datum, thuis, uit, minuut, "doelpunt", naam])

    for speler in eigen_doelpunt_spelers:
        minuut = get_minuut(speler.strip().split(". ")[0])
        naam = speler.strip().split(". ")[1]
        rows.append([id, datum, thuis, uit, minuut, "eigen_doelpunt", naam])

    for speler in penalty_spelers:
        minuut = get_minuut(speler.strip().split(". ")[0])
        naam = speler.strip().split(". ")[1]
        rows.append([id, datum, thuis, uit, minuut, "penalty", naam])


0 Rapid JC - Fortuna '54 (03-02-1957 - Voetbalstats.nl)
1 Elinkwijk - VVV (17-02-1957 - Voetbalstats.nl)
2 Sparta - Feijenoord (02-12-1956 - Voetbalstats.nl)
3 NAC - EVV Eindhoven (03-03-1957 - Voetbalstats.nl)
4 EVV Eindhoven - VVV (09-09-1956 - Voetbalstats.nl)
5 Willem II - BVV (03-02-1957 - Voetbalstats.nl)
6 Fortuna '54 - Feijenoord (07-10-1956 - Voetbalstats.nl)
7 NAC - Willem II (02-12-1956 - Voetbalstats.nl)
8 Elinkwijk - BVC Amsterdam (30-12-1956 - Voetbalstats.nl)
9 Ajax - NAC (02-09-1956 - Voetbalstats.nl)
10 Rapid JC - PSV (09-09-1956 - Voetbalstats.nl)
11 Sparta - VVV (13-01-1957 - Voetbalstats.nl)
12 SC Enschede - PSV (10-03-1957 - Voetbalstats.nl)
13 VVV - Sparta (02-06-1957 - Voetbalstats.nl)
14 Elinkwijk - BVV (20-01-1957 - Voetbalstats.nl)
15 GVAV - EVV Eindhoven (17-03-1957 - Voetbalstats.nl)
16 EVV Eindhoven - SC Enschede (19-05-1957 - Voetbalstats.nl)
17 EVV Eindhoven - BVV (30-12-1956 - Voetbalstats.nl)
18 MVV - VVV (03-02-1957 - Voetbalstats.nl)
19 DOS - Sparta (

In [101]:
df = pd.DataFrame(rows, columns=["Id", "Datum", "Thuis", "Uit", "Minuut", "Soort", "Speler"])
df["Soort"] = df["Soort"].replace("eigen_doelpunt", "eigen doelpunt")
df.to_csv('1956.csv', index=False)  

In [109]:
df.groupby(["Soort", "Thuis"])["Id"].count()

Soort           Thuis        
doelpunt        Ajax             41
                BVC Amsterdam    54
                BVV              74
                DOS              71
                EVV Eindhoven    50
                Elinkwijk        74
                Feijenoord       69
                Fortuna '54      65
                GVAV             54
                MVV              38
                NAC              50
                NOAD             54
                PSV              81
                Rapid JC         57
                SC Enschede      68
                Sparta           64
                VVV              50
                Willem II        65
eigen doelpunt  Ajax              1
                DOS               2
                Elinkwijk         2
                Feijenoord        2
                Fortuna '54       1
                GVAV              1
                MVV               1
                PSV               2
                Rapid JC          

In [26]:
rows = []

wedstrijd_URL = "https://www.voetbalstats.nl/" + wedstrijden[0]
id = ids[0]

page = requests.get(wedstrijd_URL)
soup = BeautifulSoup(page.content, "html.parser")

page_title = soup.title.string if soup.title else "No title found"
thuis = page_title.split("-")[0].strip()
uit = page_title.split("-")[1].split("(")[0].strip()
print(ids[0], page_title)

td_elements2 = soup.find_all("td", class_="linkere")
td_elements3 = soup.find_all('td', {'align': 'center', 'width': '10%', 'class': 'nl3-wizw'})

scores = []
for td in td_elements3:
    score = td.text.strip().replace(' ', '')  # Removes spaces and trims leading/trailing whitespace
    scores.append(score)


datum = ""
for td in td_elements2:
    a_tag = td.find("a", href=True)
    if a_tag and a_tag["href"].startswith("listjaarere"):
        datum = get_datum(a_tag.contents[0])


a_tags = soup.find_all("a")

doelpunt_spelers = []
penalty_spelers = []
eigen_doelpunt_spelers = []

for a_tag in a_tags:
    bal_tag = a_tag.find('img', src='./afbeeldingen/bal.jpg')
    balpen_tag = a_tag.find('img', src='./afbeeldingen/balpen.jpg')
    baled_tag = a_tag.find('img', src='./afbeeldingen/baled.jpg')
    if bal_tag:
        doelpunt_spelers.append(a_tag.contents[1])
    if balpen_tag:
        penalty_spelers.append(a_tag.contents[2])
    if baled_tag:
        eigen_doelpunt_spelers.append(a_tag.contents[2])

for speler in doelpunt_spelers:
    minuut = get_minuut(speler.strip().split(". ")[0])
    naam = speler.strip().split(". ")[1]
    rows.append([id, datum, thuis, uit, minuut, "doelpunt", naam])

for speler in eigen_doelpunt_spelers:
    minuut = get_minuut(speler.strip().split(". ")[0])
    naam = speler.strip().split(". ")[1]
    rows.append([id, datum, thuis, uit, minuut, "eigen_doelpunt", naam])

for speler in penalty_spelers:
    minuut = get_minuut(speler.strip().split(". ")[0])
    naam = speler.strip().split(". ")[1]
    rows.append([id, datum, thuis, uit, minuut, "penalty", naam])

71 NAC - Feijenoord (31-03-1957 - Voetbalstats.nl)


In [35]:
scores[1].split("\r")[0]

'0-1'